In [ ]:
import numpy as np
import 

In [ ]:
#@title Functions to estimate reliability of voxels
def vectorized_correlation(x,y):
    dim = 0

    centered_x = x - x.mean(axis=dim, keepdims=True)
    centered_y = y - y.mean(axis=dim, keepdims=True)

    covariance = (centered_x * centered_y).sum(axis=dim, keepdims=True)

    covariance = covariance / (x.shape[dim])

    x_std = x.std(axis=dim, keepdims=True)+1e-8
    y_std = y.std(axis=dim, keepdims=True)+1e-8

    corr = covariance / (x_std * y_std)

    return corr.ravel()


def get_splithalf_reliability(fmri_all_trials):
    # number of repetitions
    num_reps =  fmri_all_trials.shape[1]
    possible_inds = range(num_reps)

    # split size defined by L
    L = int(num_reps/2)
    splits_a = []

    # This for loop generates all possible ways of selecting
    # num_reps/2 indices from num_reps (=3 here)
    for subset in itertools.combinations(possible_inds, L):
        splits_a.append(np.array(subset))

    # Number of unique splits
    splits_a = splits_a[:len(splits_a)//2+1]

    # initialing rho which will store split-half correlation for all voxels
    rho = np.zeros(fmri_all_trials.shape[2])

    ## Calculating the correlation between two splits and
    ## then averaging to get noise_ceiling
    for split_a in tqdm(splits_a):
        split_b = np.delete(possible_inds, split_a)
        a = fmri_all_trials[:,split_a,:]
        b = fmri_all_trials[:,split_b,:]
        a = np.mean(a,axis=1)
        b = np.mean(b,axis=1)
        temp_corr = vectorized_correlation(a, b)
        rho += temp_corr
    rho = rho/len(splits_a)

    # Spearman - brown reliability definition from Wikipedia https://en.wikipedia.org/wiki/Spearman%E2%80%93Brown_prediction_formula
    reliability = 2*rho/(1+rho)
    return reliability

def visualize_reliability(reliability,voxel_mask):
    visual_mask_3D = np.zeros((78,93,71))
    visual_mask_3D[voxel_mask==1]= reliability
    brain_mask = '/content/example.nii'
    nii_save_path =  os.path.join('/content/reliability.nii')
    saveasnii(brain_mask,nii_save_path,visual_mask_3D)
    plotting.plot_glass_brain(nii_save_path,
                            title='Voxel reliability (Training)',plot_abs=False,
                            display_mode='lyr',colorbar=True)

In [ ]:
sub = 'sub04'  #@param ["sub01","sub02","sub03","sub04","sub05","sub06","sub07","sub08","sub09","sub10"]
ROI = "WB"

####### setting up paths to fMRI directory for given subject and ROI ######
fmri_dir = '/content/participants_data_v2021'
if ROI == "WB":
    track = "full_track"
else:
    track = "mini_track"
track_dir = os.path.join(fmri_dir, track)
sub_fmri_dir = os.path.join(track_dir, sub)
####### setting up paths to fMRI directory for given subject and ROI ######

# Wrapper to load fMRI data
fmri_train_alltrials,voxel_mask = get_fmri_all_trials(sub_fmri_dir, ROI)

# Calculate split-half reliability using training data
reliability = get_splithalf_reliability(fmri_train_alltrials)

# Reliability visualization
visualize_reliability(reliability,voxel_mask)